# <center>*Texas Department of Transportation Data Analaysis and Modeling*</center>
### <center> *Sahil Surapaneni* </center>
### <center>*7/19/19* </center>

------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------

# Contents

   #### 1 EXECUTIVE SUMMARY
   #### 2 DATA SCRAPING
       2.1 Initial Data Retrieval
       2.2 Data Update
   #### 3 DATA ANALYSIS
       3.1 Data Exploration
       3.2 Data Cleaning and Preprocessing
       3.3 Text Classification
       3.4 Model Building
   #### 3 RESULTS
   #### 4 CONCLUSION
   ------------------------------------------------------------------------------------------------------------

------------------------------------------------------------------------------------------------------------

<br></br>

# 1 EXECUTIVE SUMMARY

The TXDOT data comes from construction and maintenance projects through the Texas Department of Transportation letting records. The data was retrieved from scraping the TXDOT website, getting data since 2015 Decemeber. The scraped dataset contains 4,195 unique projects, each with a project type, county, length, time and estimates and bids for the project bidders. The scraped data is stored in an **Azure SQL Database**, along with any new data that is scraped monthly. 

The modeling portion of this project had two main goals: to be able to predict a given bidder's bid for a project, and also  which bidder is most likely to win a project. In order to model the data the text columns (bidder, type, county) had to be converted into data that can be modeled on. A simple **LabelEncoder** was used to transform bidder,county,and winning_bidder, while **Latent Dirichlet Allocation Topic Modeling** was used to classify the different project types. 

**Models Tested**: LinearRegression, SVC, SGDRegressor, RandomForestClassifier, XGBRegressor, GradientBoostingClassifier, GaussianNB, DecisionTreeRegressor, and SVR

**Model Used**: XGBRegressor was the best algorithm for both predicting bid and winning bidder

The model was able to predict the bid with **83% accuracy** and the winning/losing bidders with **99% accuracy**

<br></br>
# 2 DATA SCRAPING
<br></br>

### 2.1 Initial Data Retrieval

The Data needed for this project was retrieved from the Texas Department of Transportation Website, specifically the [Bid Tabulations and Totals Page](http://www.dot.state.tx.us/business/bt.htm).

The first challenge faced was just what data to use, the website has multiple different tables all that have there pros and cons. There is the addenda information table, data organized by CCSJ Number, tables based on county and many more. The Bid Tabulations and Totals Page had the most complete dataset yet publicly only displayed three months worth of data. Only after playing with the url was the 4 years worth of stored data displayed

In order to gain access to all this data, I needed a tool to loop though all the possible url links.
```python
base_url = 'http://www.dot.state.tx.us/insdtdot/orgchart/cmd/cserve/bidtab/'
project_type = ['sc','ll','sm'] #state construction, local letting, state maintenance
year = [str(i) for i in range(2015,dt.datetime.today().year+1)] #data from 2015 until end of current year
month = [str(i) for i in range(1,13)] #all possible months
```
To actually scrape the website and retrieve the data I used a **BeatifulSoup lxml parser** that scrapes every single possible url. 
```python
response = (requests.get(base_url+proj_type+yr+'0'+mnth+'.htm',headers = headers) if(len(mnth)<2) else requests.get(base_url+proj_type+yr+mnth+'.htm',headers = headers))
html_parser = bs(response.text , 'lxml',parse_only = only_table)
```
I am only parsing through data inside of table tags as that is where all the data is stored and it improves effeciency from just parsing through all the page data. Most of my data comes from the 'Tabs' hperlink stored in the inital table. An issue I faced is that some of the projects do not have a Tabs page, so I had to make sure I wasn't getting data from those pages.
```python
if(temp_scrape.find('title').get_text() == 'Page Not Found'):
    continue
else:
``` 
After storing all the data in an unordered table, I transfer it in a proper table, so all the data is correctly categorized. After storing all the data properly I convert it to a **pandas DataFrame** and push it to my Azure SQL Database. While pushing theb data to the database I called a stored procedure, which automatically takes the data, removes unecessary column, assigns it to the proper data type, and adds it to a new table
```python
 #Pushes Data to Database
    df.to_sql(TABLE, engine, if_exists='append')
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    cursor.execute("{call dbo.Stage_Raw_Data}")
    cursor.commit()
```

<br></br>
### 2.2 Data Update

I created a seperate IPython Notebook that essentially scrapes all the data from the last time performed. Ideally this process would be monthly as project are let month by month. In order to make sure I only scrape new data, I query my data base to get the latest 'let date' or, in essence, the last project I scraped.
```python
params = urllib.parse.quote_plus(connection_string)
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
sql = 'SELECT MAX([date_accessed]) FROM [stg].[Analysis_Ready_Data]'
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()
df = pd.read_sql(sql, conn)
```
From there the process is the same as the initial data retrieval except I change the values I use to loop through the urls to start from the latest let date. 
```python
#latest month and year
month_init = int(latest_date.month)
year_init = int(latest_date.year)

if month_init == 12:
    month_init = 1
    year_init += 1
else:
    month_init +=1

project_type = ['sc','ll','sm'] #state construction, local letting, state maintenance
year = [str(i) for i in range(year_init,dt.datetime.today().year+1)]
month = [str(i) for i in range(1,13)] #all possible months
```


<br></br>
# 3 DATA ANALYSIS
<br></br>
### 3.1 Data Exploration
Lets take a look at the basic data information

```python
df.info ()
```
```
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18952 entries, 0 to 18951
Data columns (total 16 columns):
index             18952 non-null int64
county            18952 non-null object
let_date          18952 non-null object
type              18952 non-null object
time              18952 non-null object
highway           18952 non-null object
length            18952 non-null float64
check             18952 non-null float64
misc_cost         18952 non-null float64
estimate          18952 non-null float64
bid               18952 non-null float64
over/under        18952 non-null object
bidder            18952 non-null object
winning_bidder    18952 non-null object
date_accessed     18952 non-null object
id                18952 non-null int64
dtypes: float64(5), int64(2), object(9)
memory usage: 2.3+ MB
```
The dataset I am working with is relatively small-medium sized with about 19000 rows of data. It is also important to note columns have appropriate data types and there are no null values, making this dataset pretty clean. The dataset has 16 columns, with 5 completely unrelated to modeling. 
```python
print(tabulate([['Number of Unique Counties: %d' %(df['county'].nunique())],
                ['Number of Unique Bidders: %d' %(df['bidder'].nunique())],
                ['Number of Unique Winning_Bidders: %d' %(df['winning_bidder'].nunique())],
                ['Number of Unique Project Types: %d' %(df['type'].nunique())],
                ['Number of Unique Project: %d' %(df['unique_id'].nunique())]],
               numalign="right",tablefmt="fancy_grid"))
```
```
                ╒═══════════════════════════════════════╕
                │ Number of Unique Counties: 249        │
                ├───────────────────────────────────────┤
                │ Number of Unique Bidders: 714         │
                ├───────────────────────────────────────┤
                │ Number of Unique Winning_Bidders: 544 │
                ├───────────────────────────────────────┤
                │ Number of Unique Project Types: 2159  │
                ├───────────────────────────────────────┤
                │ Number of Unique Project: 4195        │
                ╘═══════════════════════════════════════╛
```
With that table, we can see a potential issue with this dataset. The dataset only has 4,200 unique projects yet has over 2100 different project types, combine that with 714 different Bidders and this leads to a highly varied dataset. One of my main challenges was preprocessing this data so a model could accurately predict upon it. 

Let's take a look at the distribution of 'bid' and 'estimate' columns
```python
sns.distplot( df["estimate"] , color="skyblue", label="Estimate Value")
sns.distplot( df["bid"] , color="red", label="Bid Value")
plt.legend()
plt.show()
```
<img src="Pictures/bidvest.PNG" alt="Distribution Graph" width="400"/>
From the graph we can see that estimate and bid are skewed heavily to the left which could lead to issues when modeling on this data.

### 3.2 Data Cleaning and Preprocessing
I created a seperate IPython notebook to handle all the cleaning and preprocessing. In between my notebook for modeling and  my notebook for cleaning I pickle my dataframe to easily transfer data between the two notebooks. The first thing I do is store all the text columns, 'bid', and 'estimate' as the preprocessing will change all those values
```python
df_info = df_info[['county','type','bidder','winning_bidder','let_date']]
```
Next, I specify each columns datatype more specifically. I classify 'over/under','bid', and 'misc_cost' as numerics and extract just the numeric values from them. Then I create a new unique id column by combining date and seq no: as this is how it was done on the TXDOT website.

I created a new column for the lowest bid value for each unique id. I then create two new columns, **'distance_from_win'** and **'distance_from_est'**. These are the actually values that I will be predicting and then using them to determing the winning bidders and the predicted bid. I then add a proj_won column which is '1' if that bidder won that project, else it is a '0'
```python
df['distance_from_win'] = (df['bid'] - df['proj_low_bid'])/df['proj_low_bid']
df['distance_from_est'] = (df['bid'] - df['estimate'])/df['estimate']
df['proj_won'] ='default value'
win_list = []
for index,row in df.iterrows():
    if row['bidder'] == row['winning_bidder']:
        win_list.append(1)
    else:
        win_list.append(0)
df['proj_won'] = win_list
```
The next step is removing any outliers from the numeric columns. The way I decided to remove outliers is to remove any values which were **three standard deviations** away from that columns mean.
```python
df = df[((df['over/under'] - df['over/under'].mean()) / df['over/under'].std()).abs() < 3]
df = df[((df['estimate'] - df['estimate'].mean()) / df['estimate'].std()).abs() < 3]
df = df[((df['over/under'] - df['over/under'].mean()) / df['over/under'].std()).abs() < 3]
df = df[((df['misc_cost'] - df['misc_cost'].mean()) / df['misc_cost'].std()).abs() < 3]
```


### 3.3 Text Classification

Before classifying all of the text field, I needed to preprocess all the data to make sure the data is consistent. The first step in doing that is stemming all the text fields down to their base words and also removing any words that are useless for classification.

```python
#Stemming Type to just base words
stemmer = PorterStemmer()
#Checking for stopwords
words = stopwords.words("english")
#Only getting text from columns
df['type_stem'] = df['type'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())
df['bidder_stem'] = df['bidder'].apply(lambda x: " ".join([i for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())
df['winning_bidder_stem'] = df['winning_bidder'].apply(lambda x: " ".join([i for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())
df['county_stem']  = df['county'].apply(lambda x: x.lower())
```
The stopwords I am checking for are basic english stopwords in the nltk library. In the code above I am transforming all the text columns to stemmed version of non stop words comprised of only characters in the alphabet. Bidder, Winning Bidder and County are all text columns where each value in completely indpendent of each other. In that case the text classification simply involves assigning a number value to each unique value in that column using sklearn preprocessing LabelEncoder.
```python
le = preprocessing.LabelEncoder()
df['county_transform'] = le.fit_transform(df['county'])
le.fit(df['bidder_stem'])
df['bidder_transform'] = le.transform(df['bidder_stem'])
df['winning_bidder_transform'] = le.transform(df['winning_bidder_stem'])
#df = df.drop(['county','bidder','winning_bidder'])
df['distance_from_win'] = le.fit_transform(df['distance_from_win'])
```
However, for the Project Type many of the project types are similar to each other with just an extra word or two added. Since values in the column do relate to each other using a Label Encoder wouldn't be as optimal. In order to classify this text I decided to Latent Dirichlet Allocation Topic Modeling to transform each project type into the most dominant type associated with that type. The first stem in doing that is tokenizing all the project types.
```python
#Tokenizing text
def pre_processor(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if len(token) > 3:
            result.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return result

#Getting tokenized topics
topics = df['type_stem'].apply(pre_processor)
```
The next step is classfying the project types. I first created a dictionary of all the possible tokenized topics after applying the text pre preocessng. The next step is converting that dictionary into a bag of words format so it could be classified. Then I applied a TF-IDF Model on that bag of words model which essentially converts the bag of words into a value for how important it is to the overall bag of words. Finally I create a LDA Model on the TF-IDF vectorized bag of words that assigns a combination of topics to each topic that are most applicable.
```python
#Creating dictionary for topics
topic_dict = gensim.corpora.Dictionary(topics)
word_freq = [topic_dict.doc2bow(topic) for topic in topics]
tfidf = models.TfidfModel(word_freq)
corpus_tfidf = tfidf[word_freq]
optimum_num_topics = 32
# Creating LDA Model
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=optimum_num_topics, id2word=topic_dict, passes=2, workers=4)
# lda_model_tfidf.save('LDA MODEL')
```
From there I loop through the project types and assign the most dominant topic to each project type.
```python
dom_topic = []
perc_contr = []
for i in range(len(word_freq)):
    #Sorting by most contributed topic
    feature = sorted(lda_model_tfidf.get_document_topics(word_freq[i], minimum_probability=0.0), key=lambda x: (x[1]), reverse=True)[0]
    dom_topic.append(int(feature[0]))
    perc_contr.append(round(feature[1],4))
df['dominant_topic_type'] = dom_topic
df['perc_contr_type'] = perc_contr
```

### 3.4 Model Building
My goal for this project was to predict a bidder's likely bid and also whether they are likely to win a project. In order to do this, instead of predicting bid and winning bidder, I decided to predict distance from estimate and distance from win. Distance from estimate is (bid - estimate)/estimate and distance from win is (bid - lowest bid for that project)/lowest bid. As the data I am predicting is continous, it is most likely that a regression model would be the best in this case. The first thing I did is testing a bunch of different modeling algorithms to see which performed the best without paramter tuning.
```python
algorithms = [
    XGBRegressor(),
    LinearRegression(),
    #SVC(probability=True, gamma = 'auto'),
    SGDRegressor(),
    #RandomForestClassifier(),
    #GradientBoostingClassifier(),
    DecisionTreeRegressor(),
    SVR()
]
```
The one's I commented out either ran infintely or for many minutes. This makes sense as those are classification algorithms and not intended to predict continous data. These are the results for the other regression models, the main one being MAPE or Mean Average Percentage Error.
```
╒══════════════╤═════════╤════════════════════╤═══════════════╤═════════════════════════╕
│ Model Name   │    MAPE │   Total Difference │   Average DFW │   Average Predicted DFW │
╞══════════════╪═════════╪════════════════════╪═══════════════╪═════════════════════════╡
│ XGBRegressor │ 36.6637 │        3.57137e+07 │       5272.75 │                 5188.27 │
╘══════════════╧═════════╧════════════════════╧═══════════════╧═════════════════════════╛
╒══════════════╤═════════╤════════════════════╤═══════════════╤═════════════════════════╕
│ Model Name   │    MAPE │   Total Difference │   Average DFE │   Average Predicted DFE │
╞══════════════╪═════════╪════════════════════╪═══════════════╪═════════════════════════╡
│ XGBRegressor │ 145.151 │            3852.11 │      0.143654 │                0.151897 │
╘══════════════╧═════════╧════════════════════╧═══════════════╧═════════════════════════╛
╒══════════════════╤═════════╤════════════════════╤═══════════════╤═════════════════════════╕
│ Model Name       │    MAPE │   Total Difference │   Average DFW │   Average Predicted DFW │
╞══════════════════╪═════════╪════════════════════╪═══════════════╪═════════════════════════╡
│ LinearRegression │ 43.4344 │        4.23089e+07 │       5272.75 │                 5264.25 │
╘══════════════════╧═════════╧════════════════════╧═══════════════╧═════════════════════════╛
╒══════════════════╤═════════╤════════════════════╤═══════════════╤═════════════════════════╕
│ Model Name       │    MAPE │   Total Difference │   Average DFE │   Average Predicted DFE │
╞══════════════════╪═════════╪════════════════════╪═══════════════╪═════════════════════════╡
│ LinearRegression │ 152.387 │            4044.16 │      0.143654 │                0.149131 │
╘══════════════════╧═════════╧════════════════════╧═══════════════╧═════════════════════════╛
╒══════════════╤═════════════╤════════════════════╤═══════════════╤═════════════════════════╕
│ Model Name   │        MAPE │   Total Difference │   Average DFW │   Average Predicted DFW │
╞══════════════╪═════════════╪════════════════════╪═══════════════╪═════════════════════════╡
│ SGDRegressor │ 6.86494e+20 │        6.68705e+26 │       5272.75 │             2.62395e+22 │
╘══════════════╧═════════════╧════════════════════╧═══════════════╧═════════════════════════╛
╒══════════════╤════════════╤════════════════════╤═══════════════╤═════════════════════════╕
│ Model Name   │       MAPE │   Total Difference │   Average DFE │   Average Predicted DFE │
╞══════════════╪════════════╪════════════════════╪═══════════════╪═════════════════════════╡
│ SGDRegressor │ 2.2027e+24 │        5.84567e+25 │      0.143654 │            -4.47763e+20 │
╘══════════════╧════════════╧════════════════════╧═══════════════╧═════════════════════════╛
╒═══════════════════════╤═════════╤════════════════════╤═══════════════╤═════════════════════════╕
│ Model Name            │    MAPE │   Total Difference │   Average DFW │   Average Predicted DFW │
╞═══════════════════════╪═════════╪════════════════════╪═══════════════╪═════════════════════════╡
│ DecisionTreeRegressor │ 47.5628 │        4.63303e+07 │       5272.75 │                 5092.29 │
╘═══════════════════════╧═════════╧════════════════════╧═══════════════╧═════════════════════════╛
╒═══════════════════════╤═════════╤════════════════════╤═══════════════╤═════════════════════════╕
│ Model Name            │    MAPE │   Total Difference │   Average DFE │   Average Predicted DFE │
╞═══════════════════════╪═════════╪════════════════════╪═══════════════╪═════════════════════════╡
│ DecisionTreeRegressor │ 207.732 │            5512.93 │      0.143654 │                0.177816 │
╘═══════════════════════╧═════════╧════════════════════╧═══════════════╧═════════════════════════╛
╒══════════════╤═════════╤════════════════════╤═══════════════╤═════════════════════════╕
│ Model Name   │    MAPE │   Total Difference │   Average DFW │   Average Predicted DFW │
╞══════════════╪═════════╪════════════════════╪═══════════════╪═════════════════════════╡
│ SVR          │ 75.5005 │        7.35441e+07 │       5272.75 │                 4803.33 │
╘══════════════╧═════════╧════════════════════╧═══════════════╧═════════════════════════╛
╒══════════════╤═════════╤════════════════════╤═══════════════╤═════════════════════════╕
│ Model Name   │    MAPE │   Total Difference │   Average DFE │   Average Predicted DFE │
╞══════════════╪═════════╪════════════════════╪═══════════════╪═════════════════════════╡
│ SVR          │ 153.704 │            4079.09 │      0.143654 │                0.165627 │
╘══════════════╧═════════╧════════════════════╧═══════════════╧═════════════════════════╛
```
From these results we can see that Extreme Gradient Booosting Regressor (XGBRegressor) works the best for both distance from win and distance from estimate. The next step was finding the best parameters for the XGBRegressor Model. I used RandomizedSearchCV to find the optimal parameters.
``` python
params = { 
    "n_enumerators": [1,3,5,7,9],
    "n_estimators": [150, 200,250,300,350],
    "max_depth": [2, 4, 6, 8,10],
    "learning_rate": [0.01, 0.05, 0.1, 0.15, 0.2],
    "colsample_bytree": [0.3, 0.5, 0.7, 0.9, 1.0],
    "subsample":[0.3, 0.5, 0.7, 0.9, 1.0],
         }

n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=params,
                                    n_iter=n_iter_search, cv=5, iid=False)

start = time()
model_win = random_search.fit(features_win,target_win)
print(clf.__class__.__name__ + 'took ' + str(time()-start) + ' seconds')
print(random_search.best_params_)
print(random_search.best_score_)

start = time()
model_bid = random_search.fit(features_bid,target_bid)
print(clf.__class__.__name__ + 'took ' + str(time()-start) + ' seconds')
print(random_search.best_params_)
print(random_search.best_score_)
```
Using this I found the best model parameters to be
```python
optimal_win = XGBRegressor(n_jobs = 5,n_enumerators = 5,max_depth = 4, learning_rate= 0.05, n_estimators = 300)
optimal_bid = XGBRegressor(subsample= 0.9, n_estimators= 350, n_enumerators= 7, max_depth= 2, learning_rate= 0.05, colsample_bytree = 0.7)
```

<br></br>
# RESULTS
<br></br>
My results on the winning bidder model with the optimized paramters had a slightly higher MAPE. RandomizedSearchCV precicts the best parameters based on how well they should work on unseen data not just this historic dataset so it is not an issue.
```
╒═════════╤════════════════════╤════════════════════╤═══════════════════════╕
│    MAPE │   Total Difference │   Total Actual Bid │   Total Predicted Bid │
╞═════════╪════════════════════╪════════════════════╪═══════════════════════╡
│ 36.7298 │         3.5778e+07 │           97408696 │           9.55439e+07 │
╘═════════╧════════════════════╧════════════════════╧═══════════════════════╛
```
I used my model to predict distacne from win to go through all the projects and predict which bidder won or not. In order to do that I assigned a 1 to the winning bidder and 0 to the other bidders.
```
╒═════════════════════════════════╤══════════════════════╤═══════════════════════════════════════════╕
│   Number of Correct Predictions │   Number of Projects │   % of Winning Bidder Correctly Predicted │
╞═════════════════════════════════╪══════════════════════╪═══════════════════════════════════════════╡
│                            4218 │                 4269 │                                   98.8053 │
╘═════════════════════════════════╧══════════════════════╧═══════════════════════════════════════════╛
```
The distance from estimate model, on the other hand, had the MAPE significantly improve which means these paramters made a big difference in the prediction. 
```
╒═════════╤════════════════════╤════════════════════╤═══════════════════════╕
│    MAPE │   Total Differemce │   Total Actual Bid │   Total Predicted Bid │
╞═════════╪════════════════════╪════════════════════╪═══════════════════════╡
│ 94.8571 │            3877.18 │            4087.39 │                2874.6 │
╘═════════╧════════════════════╧════════════════════╧═══════════════════════╛
``` 
I used the predicted distance from win the extrapolate the predicted bid by multiplying the predicted value by estimate and adding it to estimate.
```python
tmp_df['pred_bid'] = round((tmp_df['prediction_bid'] * tmp_df['estimate']  + tmp_df['estimate']).astype(np.float64),3)
```
```
╒═════════╕
│    MAPE │
╞═════════╡
│ 18.0307 │
╘═════════╛
```
I added all my predicted values to a DataFrame which I pushed to my Azure SQL Database
<br></br>
<img src="Pictures/predict.PNG" alt="Prediction DataFrame" width="900"/>

<br></br>
# CONCLUSION
<br></br>
My models can defintely give advantages to bidders for Texas Construction Projects. This project is more of a general solution and can be used as a foundation for adding in data and information for specific companies. From this project bidders can get a better understanding of what projects it would be optimal to bid for and how much they should likely bid. 